In [2]:
import pandas as pd
import numpy as np

import pytz
import astral as ast
from astral import LocationInfo
from astral.sun import sun
from datetime import datetime, timedelta, date

In [4]:
# read in with pandas
winter1 = pd.read_csv ('gvq20-21data.txt')
winter2 = pd.read_csv ('gvq21-22data.txt')
winter3 = pd.read_csv ('gvq22-23data.txt')

In [6]:
# rename columns
winter1 = winter1.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter2 = winter2.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter3 = winter3.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})

# datetime format
winter1['datetime'] = pd.to_datetime(winter1['datetime'])
winter2['datetime'] = pd.to_datetime(winter2['datetime'])
winter3['datetime'] = pd.to_datetime(winter3['datetime'])

In [8]:
# mask out days without precipitation
codes = ['GR','GS','IC','PL','PY','RA','SG','SH','SN','SQ','TS','UP']

mask1 = winter1['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask2 = winter2['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask3 = winter3['wxcodes'].apply(lambda x: any(sub in x for sub in codes))

precip_days1 = winter1[mask1]
precip_days2 = winter2[mask2]
precip_days3 = winter3[mask3]

In [10]:
precip_days2

,station,datetime,precip(mm),wxcodes
0,GVQ,2021-12-01 00:46:00,T,-RA
4,GVQ,2021-12-01 01:38:00,T,-RA
5,GVQ,2021-12-01 01:56:00,T,UP
6,GVQ,2021-12-01 02:17:00,T,-RA
46,GVQ,2021-12-02 13:23:00,T,RA
...,...,...,...,...
3428,GVQ,2022-02-27 22:56:00,T,-SN
3429,GVQ,2022-02-27 23:36:00,T,-SN
3430,GVQ,2022-02-27 23:45:00,T,-SN
3431,GVQ,2022-02-27 23:50:00,T,-SN


In [12]:
# replace Trace with 0.0001
precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
precip_days3['precip(mm)'] = precip_days3['precip(mm)'].replace('T', 0.0001)

C:\Users\alyha\AppData\Local\Temp\ipykernel_20936\3060002225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_20936\3060002225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_20936\3060002225.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [14]:
# make sure numeric form
precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
precip_days3['precip(mm)'] = pd.to_numeric(precip_days3['precip(mm)'])

C:\Users\alyha\AppData\Local\Temp\ipykernel_20936\2630444437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_20936\2630444437.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_20936\2630444437.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [17]:
# eliminate special weather 
precip_days1 = precip_days1[precip_days1['datetime'].dt.minute == 56]
precip_days2 = precip_days2[precip_days2['datetime'].dt.minute == 56]
precip_days3 = precip_days3[precip_days3['datetime'].dt.minute == 56]

In [19]:
precip_days1.dtypes

station               object
datetime      datetime64[ns]
precip(mm)           float64
wxcodes               object
dtype: object

In [21]:
precip_days1['date'] = precip_days1['datetime'].dt.date

In [24]:
gvq = LocationInfo(
    'Batavia', 'New York', 
    latitude=42.9981,
    longitude=-78.1875, 
    timezone='America/New_York'
)

gvq

LocationInfo(name='Batavia', region='New York', timezone='America/New_York', latitude=42.9981, longitude=-78.1875)

In [27]:
# create a date range (winter 1)
date_range1 = pd.date_range(start='2020-12-01', end='2021-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times1 = []
sunset_times1 = []

# Calculate sunrise and sunset for each date
for date in date_range1:
    s = sun(gvq.observer, date=date)
    
    sunrise_times1.append(s['sunrise'])
    sunset_times1.append(s['sunset'])

# df with results
sun_times1 = pd.DataFrame({
    'datetime': date_range1.date,
    'sunrise': sunrise_times1,
    'sunset': sunset_times1
})

sun_times1['sunrise'] = pd.to_datetime(sun_times1['sunrise'])
sun_times1['sunset'] = pd.to_datetime(sun_times1['sunset'])

In [29]:
# create a date range (winter 2)
date_range2 = pd.date_range(start='2021-12-01', end='2022-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times2 = []
sunset_times2 = []

# Calculate sunrise and sunset for each date
for date in date_range2:
    s = sun(gvq.observer, date=date)
    
    sunrise_times2.append(s['sunrise'])
    sunset_times2.append(s['sunset'])

# df with results
sun_times2 = pd.DataFrame({
    'datetime': date_range2.date,
    'sunrise': sunrise_times2,
    'sunset': sunset_times2
})

sun_times2['sunrise'] = pd.to_datetime(sun_times2['sunrise'])
sun_times2['sunset'] = pd.to_datetime(sun_times2['sunset'])

In [31]:
# create a date range (winter 3)
date_range3 = pd.date_range(start='2022-12-01', end='2023-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times3 = []
sunset_times3 = []

# Calculate sunrise and sunset for each date
for date in date_range3:
    s = sun(gvq.observer, date=date)
    
    sunrise_times3.append(s['sunrise'])
    sunset_times3.append(s['sunset'])

# df with results
sun_times3 = pd.DataFrame({
    'datetime': date_range3.date,
    'sunrise': sunrise_times3,
    'sunset': sunset_times3
})

sun_times3['sunrise'] = pd.to_datetime(sun_times3['sunrise'])
sun_times3['sunset'] = pd.to_datetime(sun_times3['sunset'])

In [33]:
# add sunrise and sunset times (winter1)
sun_times1['sunrise'] = sun_times1['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times1['sunset'] = sun_times1['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times1['date'] = sun_times1['datetime']
sun_times1 = sun_times1.drop(columns=['datetime'])

print(sun_times1)

                sunrise               sunset        date
0   2020-12-01 07:25:05  2020-12-01 16:38:43  2020-12-01
1   2020-12-02 07:26:09  2020-12-02 16:38:27  2020-12-02
2   2020-12-03 07:27:12  2020-12-03 16:38:13  2020-12-03
3   2020-12-04 07:28:13  2020-12-04 16:38:01  2020-12-04
4   2020-12-05 07:29:13  2020-12-05 16:37:52  2020-12-05
..                  ...                  ...         ...
85  2021-02-24 06:56:31  2021-02-24 17:55:52  2021-02-24
86  2021-02-25 06:54:56  2021-02-25 17:57:08  2021-02-25
87  2021-02-26 06:53:20  2021-02-26 17:58:24  2021-02-26
88  2021-02-27 06:51:43  2021-02-27 17:59:40  2021-02-27
89  2021-02-28 06:50:06  2021-02-28 18:00:55  2021-02-28

[90 rows x 3 columns]


In [35]:
# add sunrise and sunset times (winter2)

sun_times2['sunrise'] = sun_times2['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times2['sunset'] = sun_times2['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times2['date'] = sun_times2['datetime']
sun_times2 = sun_times2.drop(columns=['datetime'])

print(sun_times2)

                sunrise               sunset        date
0   2021-12-01 07:24:50  2021-12-01 16:38:47  2021-12-01
1   2021-12-02 07:25:54  2021-12-02 16:38:30  2021-12-02
2   2021-12-03 07:26:57  2021-12-03 16:38:15  2021-12-03
3   2021-12-04 07:27:58  2021-12-04 16:38:03  2021-12-04
4   2021-12-05 07:28:58  2021-12-05 16:37:54  2021-12-05
..                  ...                  ...         ...
85  2022-02-24 06:56:54  2022-02-24 17:55:33  2022-02-24
86  2022-02-25 06:55:19  2022-02-25 17:56:50  2022-02-25
87  2022-02-26 06:53:44  2022-02-26 17:58:06  2022-02-26
88  2022-02-27 06:52:07  2022-02-27 17:59:21  2022-02-27
89  2022-02-28 06:50:29  2022-02-28 18:00:37  2022-02-28

[90 rows x 3 columns]


In [37]:
# add sunrise and sunset times (winter3)

sun_times3['sunrise'] = sun_times3['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times3['sunset'] = sun_times3['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times3['date'] = sun_times3['datetime']
sun_times3 = sun_times3.drop(columns=['datetime'])

print(sun_times3)

                sunrise               sunset        date
0   2022-12-01 07:24:34  2022-12-01 16:38:51  2022-12-01
1   2022-12-02 07:25:38  2022-12-02 16:38:34  2022-12-02
2   2022-12-03 07:26:41  2022-12-03 16:38:18  2022-12-03
3   2022-12-04 07:27:43  2022-12-04 16:38:06  2022-12-04
4   2022-12-05 07:28:44  2022-12-05 16:37:55  2022-12-05
..                  ...                  ...         ...
85  2023-02-24 06:57:17  2023-02-24 17:55:15  2023-02-24
86  2023-02-25 06:55:42  2023-02-25 17:56:31  2023-02-25
87  2023-02-26 06:54:07  2023-02-26 17:57:47  2023-02-26
88  2023-02-27 06:52:30  2023-02-27 17:59:03  2023-02-27
89  2023-02-28 06:50:53  2023-02-28 18:00:18  2023-02-28

[90 rows x 3 columns]


In [39]:
# add 'date' column to all dataframes
precip_days1['date'] = precip_days1['datetime'].dt.date
precip_days2['date'] = precip_days2['datetime'].dt.date
precip_days3['date'] = precip_days3['datetime'].dt.date

C:\Users\alyha\AppData\Local\Temp\ipykernel_20936\762931898.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['date'] = precip_days2['datetime'].dt.date


In [41]:
# merge precip days and sunrise/sunset times on date 
merged1 = pd.merge(precip_days1, sun_times1, on='date')

merged1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,GVQ,2020-12-01 01:56:00,0.0001,-RA BR,2020-12-01,2020-12-01 07:25:05,2020-12-01 16:38:43
1,GVQ,2020-12-01 06:56:00,0.0001,-RA,2020-12-01,2020-12-01 07:25:05,2020-12-01 16:38:43
2,GVQ,2020-12-01 09:56:00,0.0001,UP,2020-12-01,2020-12-01 07:25:05,2020-12-01 16:38:43
3,GVQ,2020-12-01 10:56:00,0.2500,-RA,2020-12-01,2020-12-01 07:25:05,2020-12-01 16:38:43
4,GVQ,2020-12-01 11:56:00,0.0001,-SN BR,2020-12-01,2020-12-01 07:25:05,2020-12-01 16:38:43
...,...,...,...,...,...,...,...
683,GVQ,2021-02-24 19:56:00,0.0001,-RA,2021-02-24,2021-02-24 06:56:31,2021-02-24 17:55:52
684,GVQ,2021-02-26 06:56:00,0.0001,-SN,2021-02-26,2021-02-26 06:53:20,2021-02-26 17:58:24
685,GVQ,2021-02-27 10:56:00,0.0001,-RA,2021-02-27,2021-02-27 06:51:43,2021-02-27 17:59:40
686,GVQ,2021-02-27 13:56:00,0.5100,-RA,2021-02-27,2021-02-27 06:51:43,2021-02-27 17:59:40


In [43]:
# merge precip days and sunrise/sunset times on date 
merged2 = pd.merge(precip_days2, sun_times2, on='date')

merged2

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,GVQ,2021-12-01 01:56:00,0.0001,UP,2021-12-01,2021-12-01 07:24:50,2021-12-01 16:38:47
1,GVQ,2021-12-02 13:56:00,0.2500,-RA,2021-12-02,2021-12-02 07:25:54,2021-12-02 16:38:30
2,GVQ,2021-12-02 20:56:00,0.5100,VCTS -RA,2021-12-02,2021-12-02 07:25:54,2021-12-02 16:38:30
3,GVQ,2021-12-05 18:56:00,0.0001,-SN,2021-12-05,2021-12-05 07:28:58,2021-12-05 16:37:54
4,GVQ,2021-12-06 03:56:00,0.0001,-RA,2021-12-06,2021-12-06 07:29:57,2021-12-06 16:37:46
...,...,...,...,...,...,...,...
728,GVQ,2022-02-27 19:56:00,0.0001,-SN BR,2022-02-27,2022-02-27 06:52:07,2022-02-27 17:59:21
729,GVQ,2022-02-27 20:56:00,0.0001,-SN,2022-02-27,2022-02-27 06:52:07,2022-02-27 17:59:21
730,GVQ,2022-02-27 21:56:00,0.0001,-SN,2022-02-27,2022-02-27 06:52:07,2022-02-27 17:59:21
731,GVQ,2022-02-27 22:56:00,0.0001,-SN,2022-02-27,2022-02-27 06:52:07,2022-02-27 17:59:21


In [45]:
# merge precip days and sunrise/sunset times on date 
merged3 = pd.merge(precip_days3, sun_times3, on='date')

merged3

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,GVQ,2022-12-01 08:56:00,0.0001,-SN,2022-12-01,2022-12-01 07:24:34,2022-12-01 16:38:51
1,GVQ,2022-12-01 10:56:00,0.0001,-SN,2022-12-01,2022-12-01 07:24:34,2022-12-01 16:38:51
2,GVQ,2022-12-01 11:56:00,0.0001,-SN,2022-12-01,2022-12-01 07:24:34,2022-12-01 16:38:51
3,GVQ,2022-12-01 12:56:00,0.0001,-SN,2022-12-01,2022-12-01 07:24:34,2022-12-01 16:38:51
4,GVQ,2022-12-03 05:56:00,0.0001,-RA,2022-12-03,2022-12-03 07:26:41,2022-12-03 16:38:18
...,...,...,...,...,...,...,...
694,GVQ,2023-02-25 20:56:00,0.0001,-SN,2023-02-25,2023-02-25 06:55:42,2023-02-25 17:56:31
695,GVQ,2023-02-25 21:56:00,0.0001,-SN,2023-02-25,2023-02-25 06:55:42,2023-02-25 17:56:31
696,GVQ,2023-02-27 21:56:00,0.0001,-SN,2023-02-27,2023-02-27 06:52:30,2023-02-27 17:59:03
697,GVQ,2023-02-27 22:56:00,0.0001,+SN FZFG,2023-02-27,2023-02-27 06:52:30,2023-02-27 17:59:03


In [47]:
# filter out only times after sunrise & before sunset 

days1 = merged1[(merged1['datetime'] >= merged1['sunrise']) & (merged1['datetime'] <= merged1['sunset'])]
days2 = merged2[(merged2['datetime'] >= merged2['sunrise']) & (merged2['datetime'] <= merged2['sunset'])]
days3 = merged3[(merged3['datetime'] >= merged3['sunrise']) & (merged3['datetime'] <= merged3['sunset'])]


days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
2,GVQ,2020-12-01 09:56:00,0.0001,UP,2020-12-01,2020-12-01 07:25:05,2020-12-01 16:38:43
3,GVQ,2020-12-01 10:56:00,0.2500,-RA,2020-12-01,2020-12-01 07:25:05,2020-12-01 16:38:43
4,GVQ,2020-12-01 11:56:00,0.0001,-SN BR,2020-12-01,2020-12-01 07:25:05,2020-12-01 16:38:43
5,GVQ,2020-12-01 12:56:00,0.0001,-SN BR,2020-12-01,2020-12-01 07:25:05,2020-12-01 16:38:43
6,GVQ,2020-12-01 13:56:00,0.2500,-SN BR,2020-12-01,2020-12-01 07:25:05,2020-12-01 16:38:43
...,...,...,...,...,...,...,...
680,GVQ,2021-02-23 16:56:00,0.0001,-SN BR,2021-02-23,2021-02-23 06:58:06,2021-02-23 17:54:35
684,GVQ,2021-02-26 06:56:00,0.0001,-SN,2021-02-26,2021-02-26 06:53:20,2021-02-26 17:58:24
685,GVQ,2021-02-27 10:56:00,0.0001,-RA,2021-02-27,2021-02-27 06:51:43,2021-02-27 17:59:40
686,GVQ,2021-02-27 13:56:00,0.5100,-RA,2021-02-27,2021-02-27 06:51:43,2021-02-27 17:59:40


In [49]:
# filter out transition hours
days1['sunrise'] = pd.to_datetime(days1['sunrise'])
days1['sunset'] = pd.to_datetime(days1['sunset'])

days1['sunrise+1'] = days1['sunrise'] + timedelta(hours = 1)
days1['sunset-1'] = days1['sunset'] - timedelta(hours = 1)

days1 = days1[days1['datetime'] <= days1['sunset-1']]
days1 = days1[days1['datetime'] >= days1['sunrise+1']]



days2['sunrise'] = pd.to_datetime(days2['sunrise'])
days2['sunset'] = pd.to_datetime(days2['sunset'])

days2['sunrise+1'] = days2['sunrise'] + timedelta(hours = 1)
days2['sunset-1'] = days2['sunset'] - timedelta(hours = 1)

days2 = days2[days2['datetime'] <= days2['sunset-1']]
days2 = days2[days2['datetime'] >= days2['sunrise+1']]



days3['sunrise'] = pd.to_datetime(days3['sunrise'])
days3['sunset'] = pd.to_datetime(days3['sunset'])

days3['sunrise+1'] = days3['sunrise'] + timedelta(hours = 1)
days3['sunset-1'] = days3['sunset'] - timedelta(hours = 1)

days3 = days3[days3['datetime'] <= days3['sunset-1']]
days3 = days3[days3['datetime'] >= days3['sunrise+1']]

C:\Users\alyha\AppData\Local\Temp\ipykernel_20936\2882998222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunrise'] = pd.to_datetime(days1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_20936\2882998222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunset'] = pd.to_datetime(days1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_20936\2882998222.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [51]:
days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset,sunrise+1,sunset-1
2,GVQ,2020-12-01 09:56:00,0.0001,UP,2020-12-01,2020-12-01 07:25:05,2020-12-01 16:38:43,2020-12-01 08:25:05,2020-12-01 15:38:43
3,GVQ,2020-12-01 10:56:00,0.2500,-RA,2020-12-01,2020-12-01 07:25:05,2020-12-01 16:38:43,2020-12-01 08:25:05,2020-12-01 15:38:43
4,GVQ,2020-12-01 11:56:00,0.0001,-SN BR,2020-12-01,2020-12-01 07:25:05,2020-12-01 16:38:43,2020-12-01 08:25:05,2020-12-01 15:38:43
5,GVQ,2020-12-01 12:56:00,0.0001,-SN BR,2020-12-01,2020-12-01 07:25:05,2020-12-01 16:38:43,2020-12-01 08:25:05,2020-12-01 15:38:43
6,GVQ,2020-12-01 13:56:00,0.2500,-SN BR,2020-12-01,2020-12-01 07:25:05,2020-12-01 16:38:43,2020-12-01 08:25:05,2020-12-01 15:38:43
...,...,...,...,...,...,...,...,...,...
678,GVQ,2021-02-23 12:56:00,0.0001,-SN BR,2021-02-23,2021-02-23 06:58:06,2021-02-23 17:54:35,2021-02-23 07:58:06,2021-02-23 16:54:35
679,GVQ,2021-02-23 15:56:00,0.0001,-RA,2021-02-23,2021-02-23 06:58:06,2021-02-23 17:54:35,2021-02-23 07:58:06,2021-02-23 16:54:35
685,GVQ,2021-02-27 10:56:00,0.0001,-RA,2021-02-27,2021-02-27 06:51:43,2021-02-27 17:59:40,2021-02-27 07:51:43,2021-02-27 16:59:40
686,GVQ,2021-02-27 13:56:00,0.5100,-RA,2021-02-27,2021-02-27 06:51:43,2021-02-27 17:59:40,2021-02-27 07:51:43,2021-02-27 16:59:40


In [53]:
# find total precip over each year
day_total1 = days1['precip(mm)'].sum()
day_total2 = days2['precip(mm)'].sum()
day_total3 = days3['precip(mm)'].sum()

# total number of obs each year
day_rows1 = days1.shape[0]
day_rows2 = days2.shape[0]
day_rows3 = days3.shape[0]

In [55]:
day_precip_per_hour1 = day_total1/ day_rows1
day_precip_per_hour2 = day_total2/ day_rows2
day_precip_per_hour3 = day_total3/ day_rows3


In [60]:
print('2020-2021:', day_precip_per_hour1)
print('2021-2022:', day_precip_per_hour2)
print('2022-2023:', day_precip_per_hour3)

2020-2021: 0.05274244897959185
2021-2022: 0.0813452380952381
2022-2023: 0.11134471544715446


In [62]:
# filter out only times after sunset & before NEXT sunrise 

nights1 = merged1[(merged1['datetime'] < merged1['sunrise']) | (merged1['datetime'] > merged1['sunset'])]
nights2 = merged2[(merged2['datetime'] < merged2['sunrise']) | (merged2['datetime'] > merged2['sunset'])]
nights3 = merged3[(merged3['datetime'] < merged3['sunrise']) | (merged3['datetime'] > merged3['sunset'])]

nights1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,GVQ,2020-12-01 01:56:00,0.0001,-RA BR,2020-12-01,2020-12-01 07:25:05,2020-12-01 16:38:43
1,GVQ,2020-12-01 06:56:00,0.0001,-RA,2020-12-01,2020-12-01 07:25:05,2020-12-01 16:38:43
9,GVQ,2020-12-01 16:56:00,0.0001,-SN,2020-12-01,2020-12-01 07:25:05,2020-12-01 16:38:43
10,GVQ,2020-12-01 17:56:00,0.0001,-SN BR,2020-12-01,2020-12-01 07:25:05,2020-12-01 16:38:43
11,GVQ,2020-12-01 18:56:00,0.0001,-SN BR,2020-12-01,2020-12-01 07:25:05,2020-12-01 16:38:43
...,...,...,...,...,...,...,...
675,GVQ,2021-02-22 22:56:00,0.0001,-SN BR,2021-02-22,2021-02-22 06:59:39,2021-02-22 17:53:19
676,GVQ,2021-02-22 23:56:00,0.0001,UP BR,2021-02-22,2021-02-22 06:59:39,2021-02-22 17:53:19
681,GVQ,2021-02-23 17:56:00,0.2500,-SN BR,2021-02-23,2021-02-23 06:58:06,2021-02-23 17:54:35
682,GVQ,2021-02-23 18:56:00,0.0001,-RA,2021-02-23,2021-02-23 06:58:06,2021-02-23 17:54:35


In [64]:
# filter out transition hours
nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
nights1['sunset'] = pd.to_datetime(nights1['sunset'])

nights1['sunrise-1'] = nights1['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights1['sunset+1'] = nights1['sunset'] + timedelta(hours = 1)

nights1 = nights1[nights1['datetime'] >= nights1['sunset+1']]
nights1 = nights1[nights1['datetime'] <= nights1['sunrise-1']]



nights2['sunrise'] = pd.to_datetime(nights2['sunrise'])
nights2['sunset'] = pd.to_datetime(nights2['sunset'])

nights2['sunrise-1'] = nights2['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights2['sunset+1'] = nights2['sunset'] + timedelta(hours = 1)

nights2 = nights2[nights2['datetime'] >= nights2['sunset+1']]
nights2 = nights2[nights2['datetime'] <= nights2['sunrise-1']]



nights3['sunrise'] = pd.to_datetime(nights3['sunrise'])
nights3['sunset'] = pd.to_datetime(nights3['sunset'])

nights3['sunrise-1'] = nights3['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights3['sunset+1'] = nights3['sunset'] + timedelta(hours = 1)

nights3 = nights3[nights3['datetime'] >= nights3['sunset+1']]
nights3 = nights3[nights3['datetime'] <= nights3['sunrise-1']]


C:\Users\alyha\AppData\Local\Temp\ipykernel_20936\36922422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_20936\36922422.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunset'] = pd.to_datetime(nights1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_20936\36922422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [66]:
# find total precip over each year
night_total1 = nights1['precip(mm)'].sum()
night_total2 = nights2['precip(mm)'].sum()
night_total3 = nights3['precip(mm)'].sum()

# total number of obs each year
night_rows1 = nights1.shape[0]
night_rows2 = nights2.shape[0]
night_rows3 = nights3.shape[0]

In [68]:
night_precip_per_hour1 = night_total1/ night_rows1
night_precip_per_hour2 = night_total2/ night_rows2
night_precip_per_hour3 = night_total3/ night_rows3

In [70]:
print('2020-2021:', night_precip_per_hour1)
print('2021-2022:', night_precip_per_hour2)
print('2022-2023:', night_precip_per_hour3)

2020-2021: 0.04869767441860466
2021-2022: 0.058312834224598924
2022-2023: 0.20050645161290326
